In [3]:
import pandas as pd
import numpy as np

data = pd.read_csv('train.csv', index_col='Loan_ID')
data.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
Loan_ID,,,,,,,,,,,
LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [12]:
condition = (data['Gender']=='Female') & (data['Education']=='Not Graduate') & (data["Credit_History"]>0)
data.loc[condition, ["Gender","Education","Credit_History"]] #索引某行某列

,Gender,Education,Credit_History
Loan_ID,,,
LP001055,Female,Not Graduate,1.0
LP001124,Female,Not Graduate,1.0
LP001135,Female,Not Graduate,1.0
LP001177,Female,Not Graduate,1.0
LP001317,Female,Not Graduate,1.0
LP001607,Female,Not Graduate,1.0
LP002354,Female,Not Graduate,1.0
LP002445,Female,Not Graduate,1.0
LP002476,Female,Not Graduate,1.0


In [16]:
# Apply Function

def num_missing(x):
    return sum(x.isnull())

print('缺失数值的列：')
print(data.apply(num_missing))


缺失数值的列：
Gender               11
Married               0
Dependents           10
Education             0
Self_Employed        23
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            5
Loan_Amount_Term      6
Credit_History       29
Property_Area         0
dtype: int64


In [29]:
#获取众数
from scipy.stats import mode
def getMode(column):
    #众数
    mode_num = mode(data['Gender'].astype('str'))
#     print(mode_num)
    mode_num = mode_num.mode[0]
#     print('Gender列的众数是：', mode_num)
    return mode_num

In [31]:
print(getMode('Gender'))

Male


/Users/zhangjifeng/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [33]:
#缺失值填充
columns = ['Gender', 'Married', 'Self_Employed']
for col in columns:
    data[col].fillna(getMode(col), inplace=True)

/Users/zhangjifeng/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [35]:
print(data.apply(num_missing))

Gender                0
Married               0
Dependents           10
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            5
Loan_Amount_Term      6
Credit_History       29
Property_Area         0
dtype: int64


In [39]:
#数据透视表
impute_grps = data.pivot_table(values=['LoanAmount'], index=['Gender','Married','Self_Employed'])
impute_grps

LoanAmount
Gender Married Self_Employed            
Female No      Male           140.500000
               No             112.868421
               Yes            171.000000
       Yes     Male            94.000000
               No             141.708333
Male   No      Male            87.200000
               No             124.350649
               Yes            128.625000
       Yes     Male           168.533333
               No             141.834356
               Yes            153.760000

In [44]:
data.loc[data['LoanAmount'].isnull()]

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
Loan_ID,,,,,,,,,,,
LP001415,Male,Yes,1,Graduate,No,3413,4053,NaN,360.0,1.0,Semiurban
LP001542,Female,Yes,0,Graduate,No,2262,0,NaN,480.0,0.0,Semiurban
LP002057,Male,Yes,0,Not Graduate,No,13083,0,NaN,360.0,1.0,Rural
LP002360,Male,Yes,NaN,Graduate,No,10000,0,NaN,360.0,1.0,Urban
LP002593,Male,Yes,1,Graduate,No,8333,4000,NaN,360.0,1.0,Urban


In [59]:
rows = data.loc[data['LoanAmount'].isnull()].iterrows()
for row in rows:
    print(row, type(row), len(row))
    print('----------')


('LP001415', Gender                    Male
Married                    Yes
Dependents                   1
Education             Graduate
Self_Employed               No
ApplicantIncome           3413
CoapplicantIncome         4053
LoanAmount                 NaN
Loan_Amount_Term           360
Credit_History               1
Property_Area        Semiurban
Name: LP001415, dtype: object) <class 'tuple'> 2
----------
('LP001542', Gender                  Female
Married                    Yes
Dependents                   0
Education             Graduate
Self_Employed               No
ApplicantIncome           2262
CoapplicantIncome            0
LoanAmount                 NaN
Loan_Amount_Term           480
Credit_History               0
Property_Area        Semiurban
Name: LP001542, dtype: object) <class 'tuple'> 2
----------
('LP002057', Gender                       Male
Married                       Yes
Dependents                      0
Education            Not Graduate
Self_Employed          

In [60]:
for i, row in data.loc[data['LoanAmount'].isnull()].iterrows():
    ind = tuple([row['Gender'],row['Married'],row['Self_Employed']])
    print(impute_grps.loc[ind])
    data.loc[i, 'LoanAmount'] = impute_grps.loc[ind].values[0]
    
# data.loc[data['LoanAmount'].isnull()]

LoanAmount    141.834356
Name: (Male, Yes, No), dtype: float64
LoanAmount    141.708333
Name: (Female, Yes, No), dtype: float64
LoanAmount    141.834356
Name: (Male, Yes, No), dtype: float64
LoanAmount    141.834356
Name: (Male, Yes, No), dtype: float64
LoanAmount    141.834356
Name: (Male, Yes, No), dtype: float64


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
Loan_ID,,,,,,,,,,,
